### 할로윈 투자 전략
- 11월 첫날에 구매 -> 4월 마지막말에 판매
- 11월 첫날에 시작가격을 구매
- 4월 마지막 날에 종가로 판매
- 판매한 금액 / 구매한 금액
- 누적 수익률

In [25]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [70]:
# AMZN 데이터 로드
df = pd.read_csv("../../csv/AMZN.csv", index_col='Date')

In [27]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1997-05-15,2.437500,2.500000,1.927083,1.958333,1.958333,72156000
1997-05-16,1.968750,1.979167,1.708333,1.729167,1.729167,14700000
1997-05-19,1.760417,1.770833,1.625000,1.708333,1.708333,6106800
1997-05-20,1.729167,1.750000,1.635417,1.635417,1.635417,5467200
1997-05-21,1.635417,1.645833,1.375000,1.427083,1.427083,18853200


In [28]:
# index를 시계열데이터로 변경
df.index = pd.to_datetime(df.index)

In [29]:
# 2000년 11월 데이터를 출력하려면?
df.loc['2000-11'].iloc[0, 0]

np.float64(35.8125)

In [30]:
# 2001년 4월의 마지막날의 종가
df.loc['2001-04'].iloc[-1]['Close']

np.float64(15.78)

In [33]:
rtn_list = []
for i in range(2000, 2011, 1):
    # 구매를 하는 월
    buy_mon = f"{i}-11"
    sell_mon = f"{i+1}-04"
    # 구매가
    buy = df.loc[buy_mon].iloc[0, 0]
    # 판매가
    sell = df.loc[sell_mon].iloc[-1]['Close']
    rtn_list.append(sell / buy)

In [36]:
acc_rtn = 1
for i in rtn_list:
    acc_rtn *= i
acc_rtn

np.float64(2.9115898553801114)

In [53]:
# 시계열데이터에서 시간을 합
start = datetime(year = 2000, month = 11, day = 1)
start

datetime.datetime(2000, 11, 1, 0, 0)

In [40]:
# 5개월 뒤
start + relativedelta(months = 5)

datetime.datetime(2001, 4, 1, 0, 0)

In [42]:
# 시계열데이터를 문자열로 변경하는 함수? strftime('%Y-%m')
halloween_df = pd.DataFrame()

for i in range(2000, 2011):
    start = datetime(year = i, month = 11, day = 1)
    end = start + relativedelta(months = 5)
    # 시계열을 문자열로
    start = start.strftime('%Y-%m')
    end = end.strftime('%Y-%m')
    # start의 첫번쨰 index
    start_df = df.loc[start].head(1)
    end_df = df.loc[end].tail(1)
    halloween_df = pd.concat([halloween_df, start_df, end_df])

In [44]:
halloween_df = halloween_df[['Open', 'Close']]

In [48]:
# 수익률 계산
halloween_df['rtn'] = 1

# 반복문을 이용하여 수익율 계산
# 인덱스의 위치가 홀수인 경우에만 수익률이 계산
for i in range(1, len(halloween_df), 2):
    # print(i)
    # 구매한 가격
    buy = halloween_df.iloc[i-1]['Open']
    sell = halloween_df.iloc[i]['Close']
    rtn = sell / buy
    halloween_df.iloc[i, 2] = rtn

/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/2143613864.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  halloween_df['rtn'] = 1
/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/2143613864.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.4406282722513089' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  halloween_df.iloc[i, 2] = rtn


In [50]:
halloween_df['acc_rtn'] = halloween_df['rtn'].cumprod()

/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1393197676.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  halloween_df['acc_rtn'] = halloween_df['rtn'].cumprod()


In [51]:
halloween_df.tail(1)

,Open,Close,rtn,acc_rtn
Date,,,,
2011-04-29,194.380005,195.809998,1.190696,2.91159


### 할로윈 투자전략 함수화
- 매개변수 4개
    - 데이터(_df)
    - 시작년도 (_start = 2010)
    - 종료년도 (_end = datetime.now().year)
    - 월 (_mon = 11)
- 복사본 생성
- 컬럼에 Date가 존재한다면 Date를 인덱스로 변환
- 인덱스를 시계열 데이터로 변경
- 반복문을 이용하여 시작년도부터 종료년도까지 반복
    - 시작년도의 월을 포함한 시계열 데이터 생성 (buy_mon)
    - 위에서 만들어진 시계열데이터에서 5개월 뒤 라는 시계열 데이터 생성 (sell_mon)
    - buy_mon, sell_mon 시계열데이터에서 년-월의 포맷으로 문자열로 변경
    - buy_mon으로 인덱스를 필터에서 첫번째 인덱스를 추출
    - sell_mon으로 인덱스를 필터해서 마지막 인덱스를 추출
    - 나온 결과를 빈 데이터프레임에 단순 행 결합
- 수익률 계산
- 누적 수익률 계산
- 만들어진 데이터프레임과 총 누적수익률을 되돌려준다.

In [98]:
def halloween(
        _df,
        _start = 2010,
        _end = datetime.now().year,
        _mon = 11
):
    df = _df.copy()

    if 'Date' in df.columns:
        df.set_index('Date', inplace=True)
    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index, utc=True)
    # 비어있는 데이터프레임을 생성
    result = pd.DataFrame()

    # 반복문을 이용해서 _start, _end까지 반복실행하는 반복문
    for i in range(_start, _end):
        buy_mon = datetime(year = i, month = _mon, day = 1)
        sell_mon = buy_mon + relativedelta(months = 5)

        buy_mon = buy_mon.strftime('%Y-%m')
        sell_mon = sell_mon.strftime('%Y-%m')
        try:
            buy = df.loc[buy_mon].head(1)
            sell = df.loc[sell_mon].tail(1)
            result = pd.concat([result, buy, sell] , axis=0)
        except:
            break
    # 수익률 계산
    result['rtn'] = 1
    for idx in range(1, len(result), 2):
        rtn = result.iloc[idx]['Close'] / result.iloc[idx-1]['Open']
        result.iloc[idx, -1] = rtn
    # 누적수익률 계산
    result['acc_rtn'] = result['rtn'].cumprod()
    # 최종 누적수익을 변수에 저장
    acc_rtn = result.iloc[-1, -1]
    return result, acc_rtn



In [115]:
df = pd.read_csv("../../csv/MSFT.csv", index_col='Date')

In [117]:
halloween(df, _mon=2)

/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1199193655.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9091229274083455' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn


(                                 Open        High         Low       Close  \
 Date                                                                        
 2010-02-01 00:00:00+00:00   28.389999   28.480000   27.920000   28.410000   
 2010-07-30 00:00:00+00:00   25.750000   25.840000   25.350000   25.809999   
 2011-02-01 00:00:00+00:00   27.799999   28.059999   27.610001   27.990000   
 2011-07-29 00:00:00+00:00   27.520000   27.709999   27.260000   27.400000   
 2012-02-01 00:00:00+00:00   29.790001   30.049999   29.760000   29.889999   
 2012-07-31 00:00:00+00:00   29.480000   29.709999   29.330000   29.469999   
 2013-02-01 00:00:00+00:00   27.670000   28.049999   27.549999   27.930000   
 2013-07-31 00:00:00+00:00   31.969999   32.049999   31.709999   31.840000   
 2014-02-03 00:00:00+00:00   37.740002   37.990002   36.430000   36.480000   
 2014-07-31 00:00:00+00:00   43.380001   43.689999   43.080002   43.160000   
 2015-02-02 00:00:00+00:00   40.590000   41.369999   40.230000  

In [139]:
rtn_list = []
for i in range(1,13):
    hw_df, acc_rtn = halloween(df, _mon=i)
    rtn_list.append(acc_rtn)
    # print(f"{i}월 부터 6개월간 투자한 수익률 : {acc_rtn}")

max_val = max(rtn_list)
max_month = rtn_list.index(max_val) + 1
print(f"최대 수익률을 얻은 월: {max_month}월")
print(f"\n{max_month}월의 최대 수익률: {max_val}\n")

최대 수익률을 얻은 월: 10월

10월의 최대 수익률: 2.4650879004085304



/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1199193655.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7514696031525278' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1199193655.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9091229274083455' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1199193655.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8157802919708029' has dtype incompatible with int64, please explicitly cast to a compatible d

In [137]:
rtn_data = 0
for i in range(1,13):
    hw_df, acc_rtn = halloween(df, _mon=i)
    # print(f"{i}월 부터 6개월간 투자한 수익률 : {acc_rtn}")
    if acc_rtn > rtn_data:
        rtn_data = acc_rtn
        month_data = i

print(f"{month_data}월에 6개월 투자 수익률이 {rtn_data}입니다")

10월에 6개월 투자 수익률이 2.4650879004085304입니다


/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1199193655.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.7514696031525278' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1199193655.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.9091229274083455' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  result.iloc[idx, -1] = rtn
/var/folders/r7/wm80f9fd57g6h8jt36r8dgwh0000gn/T/ipykernel_5216/1199193655.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8157802919708029' has dtype incompatible with int64, please explicitly cast to a compatible d